In [2]:
import cv2 as cv
from scipy.spatial import distance
import numpy as np
from collections import OrderedDict

#please provide the paths for resources.
yolomodel = {"config_path":"configuration_files/yolo-obj.cfg",
              "model_weights_path":"configuration_files/yolo-obj_best.weights",
              "dataset_names":"configuration_files/obj.names",
              "confidence_threshold": 0.5,
              "threshold":0.3
             }
             
#video_src = "highlights.mp4"
video_src = "prova2.mp4"

class Tracker:
    def __init__(self, maxLost = 30):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.lost = OrderedDict()
        self.maxLost = maxLost

    def addObject(self, new_object_location):
        self.objects[self.nextObjectID] = new_object_location
        self.lost[self.nextObjectID] = 0
        self.nextObjectID += 1

    def removeObject(self, objectID):
        del self.objects[objectID]
        del self.lost[objectID]

    @staticmethod
    def getLocation(bounding_box):
        xlt, ylt, xrb, yrb = bounding_box
        return (int((xlt + xrb) / 2.0), int((ylt + yrb) / 2.0))

    def update(self,  detections):
        if len(detections) == 0:
            lost_ids = list(self.lost.keys())

            for objectID in lost_ids:
                self.lost[objectID] +=1
                if self.lost[objectID] > self.maxLost: self.removeObject(objectID)

            return self.objects

        new_object_locations = np.zeros((len(detections), 2), dtype="int")

        for (i, detection) in enumerate(detections): new_object_locations[i] = \
            self.getLocation(detection)

        if len(self.objects)==0:
            for i in range(0, len(detections)): self.addObject(new_object_locations[i])
        else:
            objectIDs = list(self.objects.keys())
            previous_object_locations = np.array(list(self.objects.values()))
            D = distance.cdist(previous_object_locations, new_object_locations)
            row_idx = D.min(axis=1).argsort()
            cols_idx = D.argmin(axis=1)[row_idx]
            assignedRows, assignedCols = set(), set()

            for (row, col) in zip(row_idx, cols_idx):
                if row in assignedRows or col in assignedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = new_object_locations[col]
                self.lost[objectID] = 0
                assignedRows.add(row)
                assignedCols.add(col)

            unassignedRows = set(range(0, D.shape[0])).difference(assignedRows)
            unassignedCols = set(range(0, D.shape[1])).difference(assignedCols)

            if D.shape[0]>=D.shape[1]:
                for row in unassignedRows:
                    objectID = objectIDs[row]
                    self.lost[objectID] += 1
                    if self.lost[objectID] > self.maxLost:
                        self.removeObject(objectID)
            else:
                for col in unassignedCols:
                    self.addObject(new_object_locations[col])
        return self.objects

net = cv.dnn.readNetFromDarknet(yolomodel["config_path"], yolomodel["model_weights_path"])
labels = open(yolomodel["dataset_names"]).read().strip().split("\n")
np.random.seed(12345)
layer_names = net.getLayerNames()
layer_names = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

bbox_colors = np.random.randint(0, 255, size=(len(labels), 3))
maxLost = 5
tracker = Tracker(maxLost = maxLost)
cap = cv.VideoCapture(video_src)

(H, W) = (None, None)
writer = None

def count_nonblack_np(img):
    return img.any(axis=-1).sum()

def color_detection(image, show = False): #<-- True for debugging

    boundaries = [([17, 15, 100], [50, 56, 200]), #orange
    ([0, 0, 0], [255, 255, 60])] #black
    
    i = 0
    for (lower, upper) in boundaries:
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        try:
            mask = cv.inRange(image, lower, upper)
            output = cv.bitwise_and(image, image, mask = mask)
            tot_pix = count_nonblack_np(image)
            color_pix = count_nonblack_np(output)
        except:
            print("strange things..")
            return 'not_sure'
        ratio = color_pix/tot_pix
        print("ratio is:", ratio)
        if ratio > 0.01 and i == 0:
            return 'orange'
        elif ratio > 0.01 and i == 1:
            return 'black'

        i += 1

        if show:
            cv.imshow("images", np.hstack([image, output]))
            key = cv.waitKey(30)
            if key == 27:
              cv.destroyAllWindows()
    return 'not_sure'

while(True):

    success, image = cap.read()

    if not success:
        print("error!")
        break

    if W is None or H is None: (H, W) = image.shape[:2]

    blob = cv.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections_layer = net.forward(layer_names)
    detections_bbox = []
    boxes, confidences, classIDs = [], [], []

    for out in detections_layer:
        for detection in out:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > yolomodel['confidence_threshold']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv.dnn.NMSBoxes(boxes, confidences, yolomodel["confidence_threshold"], yolomodel["threshold"])

    if len(idxs)>0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            detections_bbox.append((x, y, x+w, y+h))

            clr = [int(c) for c in bbox_colors[classIDs[i]]]

            if labels[classIDs[i]] == "P":
                color = color_detection(image[y:y+h,x:x+w])
                if color != 'not_sure':
                    if color == 'black':
                        cv.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)
                    else:
                        cv.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
            else:
                cv.rectangle(image, (x, y), (x+w, y+h), clr, 2)

            cv.putText(image, "{}: {:.4f}".format(labels[classIDs[i]], confidences[i]), (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 0.5, clr, 2)

    objects = tracker.update(detections_bbox)

    for (objectID, centroid) in objects.items():
        text = "ID {}".format(objectID)
        cv.putText(image, text, (centroid[0] - 10, centroid[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv.circle(image, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    cv.imshow("image", image)
 
    key = cv.waitKey(30)
    if key == 27:
        break

    if writer is None:
        fourcc = cv.VideoWriter_fourcc(*"MJPG")
        writer = cv.VideoWriter("result.avi", fourcc, 30, (W, H), True)

    writer.write(image)

writer.release()
cap.release()
cv.destroyWindow("image")

ratio is: 0.0014619883040935672
ratio is: 0.024366471734892786
ratio is: 0.0
ratio is: 0.0702576112412178
ratio is: 0.0
ratio is: 0.0024793388429752068
ratio is: 0.0
ratio is: 0.00922266139657444
ratio is: 0.0
ratio is: 0.0038461538461538464
ratio is: 0.0004748338081671415
ratio is: 0.11490978157644824
ratio is: 0.0005875440658049354
ratio is: 0.15276145710928318
ratio is: 0.0
ratio is: 0.005538461538461538
ratio is: 0.0
ratio is: 0.14215686274509803
ratio is: 0.0
ratio is: 0.0027137042062415195
ratio is: 0.0
ratio is: 0.13403880070546736
ratio is: 0.0022727272727272726
ratio is: 0.1125
ratio is: 0.001122334455667789
ratio is: 0.1307519640852974
ratio is: 0.0
ratio is: 0.00390625
ratio is: 0.0
ratio is: 0.07966457023060797
ratio is: 0.0
ratio is: 0.18133333333333335
ratio is: 0.0
ratio is: 0.13937621832358674
ratio is: 0.0
ratio is: 0.006440281030444965
ratio is: 0.0
ratio is: 0.007246376811594203
ratio is: 0.0
ratio is: 0.010482180293501049
ratio is: 0.0
ratio is: 0.18272727272727274


ratio is: 0.0
ratio is: 0.13625730994152047
ratio is: 0.0
ratio is: 0.007056451612903226
ratio is: 0.0
ratio is: 0.0064
ratio is: 0.0
ratio is: 0.08307692307692308
ratio is: 0.0
ratio is: 0.013198757763975156
ratio is: 0.006944444444444444
ratio is: 0.0798611111111111
ratio is: 0.0
ratio is: 0.00645682001614205
ratio is: 0.0
ratio is: 0.09528214616096208
ratio is: 0.0
ratio is: 0.13533834586466165
ratio is: 0.010526315789473684
ratio is: 0.0
ratio is: 0.1192443919716647
ratio is: 0.0
ratio is: 0.2354515050167224
ratio is: 0.001443001443001443
ratio is: 0.0962000962000962
ratio is: 0.006448412698412698
ratio is: 0.09871031746031746
ratio is: 0.004375
ratio is: 0.22625
ratio is: 0.0
ratio is: 0.007932310946589107
ratio is: 0.0
ratio is: 0.12643678160919541
ratio is: 0.0
ratio is: 0.010793650793650795
ratio is: 0.0
ratio is: 0.005
ratio is: 0.0
ratio is: 0.0920138888888889
ratio is: 0.005847953216374269
ratio is: 0.08771929824561403
ratio is: 0.0
ratio is: 0.007525083612040134
ratio is: 0

ratio is: 0.0
ratio is: 0.13527851458885942
ratio is: 0.0
ratio is: 0.06326530612244897
ratio is: 0.0029806259314456036
ratio is: 0.07302533532041729
ratio is: 0.0
ratio is: 0.11495535714285714
ratio is: 0.000975609756097561
ratio is: 0.11170731707317073
ratio is: 0.0
ratio is: 0.00468384074941452
ratio is: 0.0
ratio is: 0.00101010101010101
ratio is: 0.004166666666666667
ratio is: 0.1736111111111111
ratio is: 0.0005208333333333333
ratio is: 0.0703125
ratio is: 0.0
ratio is: 0.004201680672268907
ratio is: 0.0
ratio is: 0.0012077294685990338
strange things..
ratio is: 0.001388888888888889
ratio is: 0.29305555555555557
ratio is: 0.0
ratio is: 0.008658008658008658
ratio is: 0.0
ratio is: 0.08230088495575222
ratio is: 0.0
ratio is: 0.24571428571428572
ratio is: 0.0
ratio is: 0.07172131147540983
ratio is: 0.0
ratio is: 0.00624512099921936
ratio is: 0.0
ratio is: 0.05878378378378379
ratio is: 0.0
ratio is: 0.11315136476426799
ratio is: 0.0
ratio is: 0.13935574229691877
ratio is: 0.00821917808

ratio is: 0.0007692307692307692
ratio is: 0.08307692307692308
ratio is: 0.009467455621301775
ratio is: 0.08599605522682446
ratio is: 0.013333333333333334
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0026666666666666666
ratio is: 0.012145748987854251
ratio is: 0.000725689404934688
ratio is: 0.08780841799709724
ratio is: 0.0
ratio is: 0.003304692663582287
ratio is: 0.0
ratio is: 0.009046052631578948
ratio is: 0.0013605442176870747
ratio is: 0.2707482993197279
ratio is: 0.0
ratio is: 0.014788732394366197
ratio is: 0.0033195020746887966
ratio is: 0.2099585062240664
ratio is: 0.0
ratio is: 0.016666666666666666
ratio is: 0.0
ratio is: 0.24218891096261513
ratio is: 0.0
ratio is: 0.14381474710542352
ratio is: 0.001635322976287817
ratio is: 0.31479967293540473
ratio is: 0.0
ratio is: 0.1801441152922338
ratio is: 0.0003846153846153846
ratio is: 0.07769230769230769
ratio is: 0.0023923444976076554
ratio is: 0.09968102073365231
ratio is: 0.0012787723785166241
ratio is: 0.079923273657289
ra

ratio is: 0.0
ratio is: 0.011274509803921568
ratio is: 0.0
ratio is: 0.04922480620155039
ratio is: 0.007352941176470588
ratio is: 0.09068627450980392
ratio is: 0.0
ratio is: 0.007389162561576354
ratio is: 0.0
ratio is: 0.041304347826086954
ratio is: 0.0
ratio is: 0.0016420361247947454
ratio is: 0.0
ratio is: 0.11888544891640866
ratio is: 0.0
ratio is: 0.23203703703703704
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.13
ratio is: 0.0005194805194805195
ratio is: 0.10025974025974026
ratio is: 0.0
ratio is: 0.05555555555555555
ratio is: 0.0
ratio is: 0.17027027027027028
ratio is: 0.0
ratio is: 0.0028790786948176585
ratio is: 0.0012406947890818859
ratio is: 0.4143920595533499
ratio is: 0.0
ratio is: 0.06365740740740741
ratio is: 0.0
ratio is: 0.17810331534309945
ratio is: 0.0
ratio is: 0.3100303951367781
ratio is: 0.0015873015873015873
ratio is: 0.18412698412698414
ratio is: 0.0
ratio is: 0.012682926829268294
ratio is: 0.0
ratio is: 0.053875968992248065
ratio is: 0.0
ratio is: 0.011

ratio is: 0.023809523809523808
ratio is: 0.0
ratio is: 0.00744047619047619
ratio is: 0.0
ratio is: 0.004861111111111111
ratio is: 0.0
ratio is: 0.005580357142857143
ratio is: 0.011837121212121212
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.007763975155279503
ratio is: 0.0
ratio is: 0.044802867383512544
ratio is: 0.0
ratio is: 0.017943409247757072
ratio is: 0.0
ratio is: 0.0633903133903134
ratio is: 0.0022222222222222222
ratio is: 0.13925925925925925
ratio is: 0.0
ratio is: 0.3406113537117904
ratio is: 0.0
ratio is: 0.09779614325068871
ratio is: 0.0
ratio is: 0.285385500575374
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0031746031746031746
ratio is: 0.173015873015873
ratio is: 0.0
ratio is: 0.2916666666666667
ratio is: 0.0
ratio is: 0.21732418524871355
ratio is: 0.01596467391304348
ratio is: 0.010237510237510237
ratio is: 0.0
ratio is: 0.006947890818858561
ratio is: 0.0
ratio is: 0.0058953574060427415
ratio is: 0.0
ratio is: 0.005847953216374269
ratio is: 0.0
ratio is: 0.009259259

ratio is: 0.01107419712070875
ratio is: 0.014354066985645933
ratio is: 0.0
ratio is: 0.0922722029988466
ratio is: 0.0
ratio is: 0.012899262899262898
ratio is: 0.0
ratio is: 0.006060606060606061
ratio is: 0.0
ratio is: 0.29411764705882354
ratio is: 0.0
ratio is: 0.009565217391304347
ratio is: 0.0
ratio is: 0.1457000710732054
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.06694214876033058
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.13322368421052633
ratio is: 0.0
ratio is: 0.11475409836065574
ratio is: 0.0
ratio is: 0.11002921129503408
ratio is: 0.0
ratio is: 0.23507893723527146
ratio is: 0.0
ratio is: 0.2925170068027211
ratio is: 0.017142857142857144
ratio is: 0.013806706114398421
ratio is: 0.0
ratio is: 0.08213096559378469
ratio is: 0.0
ratio is: 0.015873015873015872
ratio is: 0.0
ratio is: 0.006274509803921568
ratio is: 0.0
ratio is: 0.13690851735015772
ratio is: 0.0
ratio is: 0.0049261083743842365
ratio is: 0.0
ratio is: 0.060528559249786874
ratio is: 0.0
ratio is: 

ratio is: 0.0
ratio is: 0.008301404853128991
ratio is: 0.01824212271973466
ratio is: 0.0
ratio is: 0.16050156739811913
ratio is: 0.0
ratio is: 0.008771929824561403
ratio is: 0.0017045454545454545
ratio is: 0.053409090909090906
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.0
ratio is: 0.008620689655172414
ratio is: 0.09985632183908046
ratio is: 0.0
ratio is: 0.015235457063711912
ratio is: 0.007222824124232575
ratio is: 0.058866016612495485
ratio is: 0.0
ratio is: 0.11401098901098901
ratio is: 0.0
ratio is: 0.1111111111111111
ratio is: 0.0
ratio is: 0.11214953271028037
ratio is: 0.0019157088122605363
ratio is: 0.07183908045977011
ratio is: 0.0014204545454545455
ratio is: 0.15198863636363635


KeyboardInterrupt: 